In [ ]:
import erlab.io

erlab.io.loaders

In [ ]:
%config InlineBackend.figure_formats = ["svg", "pdf"]
import xarray as xr
import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 96
plt.rcParams["image.cmap"] = "viridis"

xr.set_options(display_expand_data=False)
nb_execution_mode = "cache"

In [ ]:
erlab.io.loaders["merlin"]
erlab.io.loaders["bl403"]
erlab.io.loaders.merlin
erlab.io.loaders.bl403

In [ ]:
import os

import pandas as pd
from erlab.io.dataloader import LoaderBase


class MyLoader(LoaderBase):
    name = "my_loader"
    aliases = None
    name_map = {}
    coordinate_attrs = {}
    additional_attrs = {"information": "any metadata you want to load with the data"}
    skip_validate = False
    always_single = True

    def identify(self, num, data_dir):
        file = os.path.join(data_dir, f"data_{str(num).zfill(4)}.csv")
        return [file], {}

    def load_single(self, file_path):
        return pd.read_csv(file_path).to_xarray()

In [ ]:
erlab.io.loaders

In [ ]:
erlab.io.loaders["my_loader"]

In [ ]:
import csv
import datetime
import os
import tempfile

import erlab.io
import numpy as np
from erlab.io.exampledata import generate_data_angles


def make_data(beta=5.0, temp=20.0, hv=50.0, bandshift=0.0):
    data = generate_data_angles(
        shape=(250, 1, 300),
        angrange={"alpha": (-15, 15), "beta": (beta, beta)},
        hv=hv,
        configuration=1,
        temp=temp,
        bandshift=bandshift,
        assign_attributes=False,
        seed=1,
    ).T

    # Rename coordinates. The loader must rename them back to the original names.
    data = data.rename(
        {
            "alpha": "ThetaX",
            "beta": "Polar",
            "eV": "BindingEnergy",
            "hv": "PhotonEnergy",
            "xi": "Tilt",
            "delta": "Azimuth",
        }
    )

    # Assign some attributes that real data would have
    data = data.assign_attrs(
        {
            "LensMode": "Angular30",  # Lens mode of the analyzer
            "SpectrumType": "Fixed",  # Acquisition mode of the analyzer
            "PassEnergy": 10,  # Pass energy of the analyzer
            "UndPol": 0,  # Undulator polarization
            "DateTime": datetime.datetime.now().isoformat(),  # Acquisition time
            "TB": temp,
            "X": 0.0,
            "Y": 0.0,
            "Z": 0.0,
        }
    )
    return data


# Create a temporary directory
tmp_dir = tempfile.TemporaryDirectory()

# Define coordinates for the scan
beta_coords = np.linspace(2, 7, 10)

# Generate and save cuts with different beta values
for i, beta in enumerate(beta_coords):
    erlab.io.save_as_hdf5(
        make_data(beta=beta, temp=20.0, hv=50.0),
        filename=f"{tmp_dir.name}/data_001_S{str(i+1).zfill(3)}.h5",
        igor_compat=False,
    )

# Write scan coordinates to a csv file
with open(f"{tmp_dir.name}/data_001_axis.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Index", "Polar"])

    for i, beta in enumerate(beta_coords):
        writer.writerow([i + 1, beta])

# Generate some cuts with different band shifts
for i in range(4):
    erlab.io.save_as_hdf5(
        make_data(beta=5.0, temp=20.0, hv=50.0, bandshift=-i * 0.05),
        filename=f"{tmp_dir.name}/data_{str(i+2).zfill(3)}.h5",
        igor_compat=False,
    )

# List the generated files
sorted(os.listdir(tmp_dir.name))

In [ ]:
import glob
import re

import pandas as pd
from erlab.io.dataloader import LoaderBase


class ExampleLoader(LoaderBase):
    name = "example"

    aliases = ["Ex"]

    name_map = {
        "eV": "BindingEnergy",
        "alpha": "ThetaX",
        "beta": [
            "Polar",
            "Polar Compens",
        ],  # Can have multiple names assigned to the same name
        "delta": "Azimuth",
        "xi": "Tilt",
        "x": "X",
        "y": "Y",
        "z": "Z",
        "hv": "PhotonEnergy",
        "polarization": "UndPol",
        "temp_sample": "TB",
    }

    coordinate_attrs: tuple[str, ...] = (
        "beta",
        "delta",
        "xi",
        "hv",
        "x",
        "y",
        "z",
        "polarization",
        "photon_flux",
    )
    # Attributes to be used as coordinates. Place all attributes that we don't want to
    # lose when merging multiple file scans here.

    additional_attrs = {
        "configuration": 1,  # Experimental geometry. Required for momentum conversion
        "sample_workfunction": 4.3,
    }
    # Any additional metadata you want to add to the data. Note that attributes defined
    # here will not be transformed into coordinates. If you wish to promote some fixed
    # attributes to coordinates, add them to additional_coords.

    additional_coords = {}
    # Additional non-dimension coordinates to be added to the data, for instance the
    # photon energy for lab-based ARPES.

    skip_validate = False

    always_single = False

    def identify(self, num, data_dir):
        coord_dict = {}

        # Look for scans with data_###_S###.h5, and sort them
        files = glob.glob(f"data_{str(num).zfill(3)}_S*.h5", root_dir=data_dir)
        files.sort()

        if len(files) == 0:
            # If no files found, look for data_###.h5
            files = glob.glob(f"data_{str(num).zfill(3)}.h5", root_dir=data_dir)

        else:
            # If files found, extract coordinate values from the filenames
            axis_file = f"{data_dir}/data_{str(num).zfill(3)}_axis.csv"
            with open(axis_file) as f:
                header = f.readline().strip().split(",")

            # Load the coordinates from the csv file
            coord_arr = np.loadtxt(axis_file, delimiter=",", skiprows=1)

            # Each header entry will contain a dimension name
            for i, hdr in enumerate(header[1:]):
                key = self.name_map_reversed.get(hdr, hdr)
                coord_dict[key] = coord_arr[: len(files), i + 1].astype(np.float64)

        if len(files) == 0:
            # If no files found up to this point, raise an error
            raise FileNotFoundError(f"No files found for scan {num} in {data_dir}")

        # Files must be full paths
        files = [os.path.join(data_dir, f) for f in files]

        return files, coord_dict

    def load_single(self, file_path):
        data = erlab.io.load_hdf5(file_path)

        # To prevent conflicts when merging multiple scans, we rename the coordinates
        # prior to concatenation
        return self.process_keys(data)

    def infer_index(self, name):
        # Get the scan number from file name
        try:
            scan_num: str = re.match(r".*?(\d{3})(?:_S\d{3})?", name).group(1)
        except (AttributeError, IndexError):
            return None, None

        if scan_num.isdigit():
            # The second return value, a dictionary, is reserved for more complex
            # setups. See tips below for a brief explanation.
            return int(scan_num), {}
        else:
            return None, None

In [ ]:
erlab.io.loaders

In [ ]:
erlab.io.set_loader("example")
erlab.io.set_data_dir(tmp_dir.name)
erlab.io.load(1)

In [ ]:
erlab.io.load(5).qplot()

In [ ]:
class ExampleLoaderComplete(ExampleLoader):
    name = "example_complete"
    aliases = ["ExC"]

    def generate_summary(self, data_dir):
        # Get all valid data files in directory
        files = {}
        for path in erlab.io.utils.get_files(data_dir, extensions=[".h5"]):
            # Base name
            data_name = os.path.splitext(os.path.basename(path))[0]

            # If multiple scans, strip the _S### part
            name_match = re.match(r"(.*?_\d{3})_(?:_S\d{3})?", data_name)
            if name_match is not None:
                data_name = name_match.group(1)

            files[data_name] = path

        # Map dataframe column names to data attributes
        attrs_mapping = {
            "Lens Mode": "LensMode",
            "Scan Type": "SpectrumType",
            "T(K)": "temp_sample",
            "Pass E": "PassEnergy",
            "Polarization": "polarization",
            "hv": "hv",
            "x": "x",
            "y": "y",
            "z": "z",
            "polar": "beta",
            "tilt": "xi",
            "azi": "delta",
        }
        column_names = ["File Name", "Path", "Time", "Type", *attrs_mapping.keys()]

        data_info = []

        processed_indices = set()
        for name, path in files.items():
            # Skip already processed multi-file scans
            index, _ = self.infer_index(name)
            if index in processed_indices:
                continue
            elif index is not None:
                processed_indices.add(index)

            # Load data
            data = self.load(path)

            # Determine type of scan
            data_type = "core"
            if "alpha" in data.dims:
                data_type = "cut"
            if "beta" in data.dims:
                data_type = "map"
            if "hv" in data.dims:
                data_type = "hvdep"

            data_info.append(
                [
                    name,
                    path,
                    datetime.datetime.fromisoformat(data.attrs["DateTime"]),
                    data_type,
                ]
            )

            for k, v in attrs_mapping.items():
                # Try to get the attribute from the data, then from the coordinates
                try:
                    val = data.attrs[v]
                except KeyError:
                    try:
                        val = data.coords[v].values
                        if val.size == 1:
                            val = val.item()
                    except KeyError:
                        val = ""

                # Convert polarization values to human readable form
                if k == "Polarization":
                    if np.iterable(val):
                        val = np.asarray(val).astype(int)
                    else:
                        val = [round(val)]
                    val = [{0: "LH", 2: "LV", -1: "RC", 1: "LC"}.get(v, v) for v in val]
                    if len(val) == 1:
                        val = val[0]

                data_info[-1].append(val)

            del data

        # Sort by time and set index
        return (
            pd.DataFrame(data_info, columns=column_names)
            .sort_values("Time")
            .set_index("File Name")
        )
    
erlab.io.loaders

In [ ]:
erlab.io.set_loader("example_complete")
erlab.io.summarize()